In [1]:
import time

import gurobipy as gp
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib_inline
import numpy as np
import pandas as pd
from gurobipy import GRB
from tqdm import tqdm

from orc.primal import dobson, greedy, hall_hochbaum
from orc.relaxation import subgrad_opt, lp_rel
from orc.utils import generate_problem

mpl.rcParams["font.family"] = ["serif"]
mpl.rcParams["font.serif"] = ["cmr10"]
matplotlib_inline.backend_inline.set_matplotlib_formats("retina")

# Primal heuristics comparisons

## Without fixed variables

In [ ]:
table = []
funcs = [(greedy, "Greedy"), (dobson, "Dobson"),
         (hall_hochbaum, "Hall-Hochbaum")]
for rows, cols in tqdm([(5, 10), (10, 20), (20, 50), (50, 100)]):
    res = {"Rows": rows, "Cols": cols, "Greedy": 0, 
           "Dobson": 0, "Hall-Hochbaum": 0}
    for i in tqdm(range(10)):
        A, b = generate_problem(rows, cols)
        v = []
        for f, name in funcs:
            ub = np.sum(A, axis=0) @ f(A, b, [], [])
            v.append(ub)
        j = np.argmin(v)
        best = funcs[j][1]
        res[best] += 1
    table.append(res)

In [ ]:
table

In [ ]:
df = pd.DataFrame(table)
df = df.set_index(["Rows", "Cols"])
df.style.to_latex(column_format="llccc")

## With fixed variables

In [ ]:
np.random.seed(42)

table = []
funcs = [(greedy, "Greedy"), (dobson, "Dobson"),
         (hall_hochbaum, "Hall-Hochbaum")]
for rows, cols in tqdm([(5, 10), (10, 20), (20, 50), (50, 100)]):
    res = {"Rows": rows, "Cols": cols, "Greedy": 0, 
           "Dobson": 0, "Hall-Hochbaum": 0}
    for i in tqdm(range(10)):
        A, b = generate_problem(rows, cols)
        fixed_n = int(cols * 0.3) 
        fixed = np.random.choice(
            A.shape[-1], fixed_n, replace=False)
        x0 = fixed[:int(fixed_n / 2)]
        x1 = fixed[int(fixed_n / 2) + 1:]
        x = np.ones(A.shape[-1])
        x[x0] = 0
        if np.any(A @ x < b):
            continue
        v = []
        for f, name in funcs:
            ub = np.sum(A, axis=0) @ f(A, b, [], [])
            v.append(ub)
        j = np.argmin(v)
        best = funcs[j][1]
        res[best] += 1
    table.append(res)

In [ ]:
table

In [ ]:
df = pd.DataFrame(table)
df = df.set_index(["Rows", "Cols"])
df.style.to_latex(column_format="llccc")

# Subgradient optimization lower bounds

In [ ]:
data = {}
for rows, cols in tqdm([(10, 20), (20, 50), (40, 80), (80, 150)]):
    A, b = generate_problem(rows, cols)
    x = hall_hochbaum(A, b, [], [])
    ub = np.sum(A, axis=0) @ x

    res = []
    for omega in [10, 20, 40, 50, 70, 100, 200, 500]:
        lb = subgrad_opt(A, b, ub, [], [], omega=omega)
        res.append([omega, lb])
    data[(rows, cols)] = res

In [ ]:
fig, ax = plt.subplots()
for k, v in data.items():
    x = np.array(v)[:,0]
    y = np.array(v)[:,1]
    y = (y - y.min()) / (y.max() - y.min())
    ax.plot(x, y, label=f"{k[0]} rows, {k[1]} cols")
ax.legend()
ax.set_xlabel("Iterations")
ax.set_ylabel("Scaled lower bound")
ax.set_title("Subgradient optimization lower bounds")

# Branch and Bound models

In [2]:
from orc.branch_bound import (
    BranchAndBound,
    branch_strategy, branch_strategy2, branch_strategy3
)
from orc.callbacks import (
    ColumnInclusionCallback,
    LagrPenaltiesReductionCallback,
    PrimalHeurCallback
)

In [3]:
lagr_callback = LagrPenaltiesReductionCallback()
col_callback = ColumnInclusionCallback()
primal_heur = PrimalHeurCallback()
primal_heur_root = PrimalHeurCallback(only_root=True)

models = {
    "Subgrad": {
        "branch_strategy": branch_strategy, 
        "lb_strategy": subgrad_opt
        },
    "SubgradPrimal": {
        "branch_strategy": branch_strategy, 
        "lb_strategy": subgrad_opt,
        "callbacks": [primal_heur]
        },
    "SubgradPrimalRed": {
        "branch_strategy": branch_strategy, 
        "lb_strategy": subgrad_opt,
        "callbacks": [primal_heur, lagr_callback, col_callback]
        },
    "SubgradPrimalRootRed": {
        "branch_strategy": branch_strategy, 
        "lb_strategy": subgrad_opt,
        "callbacks": [primal_heur, lagr_callback, col_callback]
        },
    "SubgradPrimalRedBeasleyBranch": {
        "branch_strategy": branch_strategy3, 
        "lb_strategy": subgrad_opt,
        "callbacks": [primal_heur, lagr_callback, col_callback]
        },
    "LPPrimalRed": {
        "branch_strategy": branch_strategy2, 
        "lb_strategy": lp_rel,
        "callbacks": [primal_heur, col_callback]}
}

In [7]:
data = {}
for rows, cols, density in tqdm([(5, 10, 0.3), (5, 10, 0.5), (5, 10, 0.7), 
                        (10, 20, 0.3), (10, 20, 0.5), (10, 20, 0.7)
                        ]):
    res = {}
    A, b = generate_problem(rows, cols, density)
    
    start = time.process_time()
    m = gp.Model()
    x = m.addMVar(A.shape[-1], vtype=GRB.BINARY, name="x")
    m.setObjective(np.sum(A, axis=0) @ x)
    m.addConstr(A @ x >= b)
    m.optimize()
    elapsed = time.process_time() - start
    opt_gurobi = m.getObjective().getValue()
    
    x = []
    for v in m.getVars():
        x.append(v.x)
    x = np.array(x)
    
    # Make sure that the solution is feasible
    assert np.all(A @ x >= b)

    res["Gurobi"] = (elapsed, np.nan)

    for model, config in tqdm(models.items()):
        start = time.process_time()
        bb = BranchAndBound(**config)
        bb.search(A, b)
        elapsed = time.process_time() - start
        x = np.zeros(A.shape[-1])
        x[bb.best.x1] = 1
        
        # Make sure that the solution is feasible
        assert np.all(A @ x >= b)
        opt = np.sum(A, axis=0) @ x

        # Check that the solution is optimal
        assert opt == opt_gurobi

        res[model] = (elapsed, bb.node_count)

    data[(rows, cols, density)] = res

  0%|          | 0/6 [00:00<?, ?it/s]

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9700K CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 10 columns and 15 nonzeros
Model fingerprint: 0xe44abcae
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [4e+01, 1e+02]
  Objective range  [7e+01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+01, 9e+01]
Found heuristic solution: objective 684.0000000
Presolve removed 5 rows and 10 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 450 684 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.500000000000e+02, best bound 4.500000000000e+02, gap 0.0000%


e:\projects\orc\orc\relaxation.py:18: RuntimeWarning: invalid value encountered in matmul
  rc = (1 - lambd) @ A
e:\projects\orc\orc\relaxation.py:36: RuntimeWarning: invalid value encountered in add
  np.zeros_like(lambd), lambd + sigma * g)
e:\projects\orc\orc\relaxation.py:22: RuntimeWarning: invalid value encountered in matmul
  L = rc @ x + lambd @ b
 17%|█▋        | 1/6 [00:00<00:03,  1.65it/s]

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9700K CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 10 columns and 25 nonzeros
Model fingerprint: 0xa276b143
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+01, 5e+01]
  Objective range  [3e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 7e+01]
Found heuristic solution: objective 399.0000000
Presolve time: 0.00s
Presolved: 5 rows, 10 columns, 25 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)
Found heuristic solution: objective 351.0000000

Root relaxation: objective 2.873991e+02, 9 iterations, 0.02 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  287.39906    0

e:\projects\orc\orc\relaxation.py:36: RuntimeWarning: invalid value encountered in multiply
  np.zeros_like(lambd), lambd + sigma * g)
 33%|███▎      | 2/6 [00:03<00:07,  1.99s/it]

...

 67%|██████▋   | 4/6 [00:53<00:40, 20.43s/it]

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9700K CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 20 columns and 100 nonzeros
Model fingerprint: 0x27f765ec
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [5e+00, 2e+02]
  Objective range  [1e+02, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 3e+02]
Found heuristic solution: objective 2857.0000000
Presolve time: 0.00s
Presolved: 10 rows, 20 columns, 100 nonzeros
Variable types: 0 continuous, 20 integer (20 binary)
Found heuristic solution: objective 2698.0000000

Root relaxation: objective 2.012073e+03, 15 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2012.073

 83%|████████▎ | 5/6 [05:28<01:52, 112.20s/it]

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9700K CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 20 columns and 140 nonzeros
Model fingerprint: 0xc03cd04b
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [3e+00, 1e+02]
  Objective range  [2e+02, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 4e+02]
Found heuristic solution: objective 3294.0000000
Presolve time: 0.00s
Presolved: 10 rows, 20 columns, 140 nonzeros
Variable types: 0 continuous, 20 integer (20 binary)
Found heuristic solution: objective 2444.0000000

Root relaxation: objective 2.168855e+03, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2168.854

100%|██████████| 6/6 [13:34<00:00, 135.81s/it]


In [8]:
data

{(5, 10, 0.3): {'Gurobi': (0.0, nan),
  'Subgrad': (0.03125, 81),
  'SubgradPrimal': (0.03125, 45),
  'SubgradPrimalRed': (0.046875, 37),
  'SubgradPrimalRootRed': (0.03125, 37),
  'SubgradPrimalRedBeasleyBranch': (0.046875, 39),
  'LPPrimalRed': (0.046875, 171)},
 (5, 10, 0.5): {'Gurobi': (0.015625, nan),
  'Subgrad': (0.203125, 443),
  'SubgradPrimal': (0.265625, 317),
  'SubgradPrimalRed': (0.171875, 207),
  'SubgradPrimalRootRed': (0.140625, 207),
  'SubgradPrimalRedBeasleyBranch': (0.234375, 207),
  'LPPrimalRed': (0.234375, 585)},
 (5, 10, 0.7): {'Gurobi': (0.015625, nan),
  'Subgrad': (0.0625, 129),
  'SubgradPrimal': (0.0625, 45),
  'SubgradPrimalRed': (0.03125, 35),
  'SubgradPrimalRootRed': (0.046875, 35),
  'SubgradPrimalRedBeasleyBranch': (0.0625, 35),
  'LPPrimalRed': (0.21875, 667)},
 (10, 20, 0.3): {'Gurobi': (0.046875, nan),
  'Subgrad': (3.984375, 5683),
  'SubgradPrimal': (1.296875, 1951),
  'SubgradPrimalRed': (1.15625, 697),
  'SubgradPrimalRootRed': (1.296875, 697)

In [12]:
import json
with open('data.json', 'w') as f:
    nd = {str(k): v for k, v in data.items()}
    json.dump(nd, f)

In [ ]:
[(5, 10, 0.3), (5, 10, 0.5), (5, 10, 0.7), 
                        (10, 20, 0.3), (10, 20, 0.5), (10, 20, 0.7), 
                        (20, 50, 0.3), (20, 50, 0.5), (20, 50, 0.7),
                        (50, 100, 0.3), (50, 100, 0.5), (50, 100, 0.7)]